In [2]:
import gradio as gr
from openai import OpenAI

class ChatBot:
    def __init__(self):
        """Initialize the ChatBot with an empty message list and no client."""
        self.messages = []  # List to store chat messages
        self.client = None  # OpenAI client will be initialized later
    
    def moderator(self, messages):
        """Check content against OpenAI's moderation endpoint.

        Args:
            messages (str): The message content to be checked.

        Returns:
            bool: True if the content is flagged, False otherwise.
        """
        try:
            response = self.client.moderations.create(
                model="omni-moderation-latest",
                input=messages,
            )
            return response.results[0].flagged  # Return whether the message is flagged
            
        except Exception as e:
            print(f"Moderation error: {str(e)}")  # Log any moderation errors
            return False  # Return False if an error occurs
        
    def initialize_bot(self, api_key):
        """Initialize the OpenAI client with the provided API key and set up the initial system message.

        Args:
            api_key (str): The API key for authenticating with the OpenAI service.

        Returns:
            str: A message indicating the success or failure of the bot initialization.
        """
        try:
            self.client = OpenAI(api_key=api_key)  # Initialize OpenAI client with the API key
            
            # Initialize system message with instructions for the chatbot
            self.messages = [{
                "role": "system",
                "content": """
                You are a helpful and context-aware technical recruiter chatbot designed to conduct initial screening interviews. Your goal is to assess a candidate's technical skills based on their declared tech stack.

                **Conversation Flow:**

                1.  **Greeting and Purpose:** Begin with a professional greeting, such as "Welcome to the technical screening interview. I'm here to assess your technical skills for potential roles. This interview will cover your background, experience, and technical expertise."
                2.  **Intent:** Ask the questions in a natural and conversational manner to make the candidate feel comfortable and engaged. You also need to make sure flow of the conversation is smooth and the candidate has a seemless experience.
                3.  **Essential Information:** Ask the following questions one by one, waiting for the candidate's response before proceeding:
                    *   Full Name
                    *   Email Address
                    *   Phone Number
                    *   Years of Experience
                    *   Desired Position(s)
                    *   Current Location
                4.  **Tech Stack Declaration:** Ask the candidate to list their proficient technologies, including Programming Languages, Frameworks, Databases, Tools, and other relevant skills. Be specific: "Please list the programming languages, frameworks, databases, and tools you are proficient in. For example, Python, Django, PostgreSQL, Git, etc."
                5.  **Technical Question Generation (3-5 questions *per technology*):** Based on the declared tech stack, generate 3-5 technical questions *per technology* to assess proficiency. For example, if the candidate lists Python and Django, generate 3-5 questions for both Python and Django. Add some questions which are linked to these technologies.
                    Adjust the difficulty based on years of experience:
                    *   0-2 years: Focus on basic concepts and syntax. (e.g., "What is the difference between a list and a tuple in Python?")
                    *   3-5 years: Focus on intermediate applications and common use cases. (e.g., "Explain the purpose of middleware in Django.")
                    *   5+ years: Focus on advanced scenarios, design patterns, and performance optimization. (e.g., "Describe a strategy for optimizing database queries in a high-traffic Django application.")
                    Use a variety of question types, including:
                        *   Conceptual questions (e.g., "What is polymorphism?")
                        *   Coding challenges (e.g., "Write a function to reverse a string.")
                        *   Debugging scenarios (e.g., "Given this code snippet, what is the likely cause of the error?")
                        *   Application scenarios (e.g., "How would you design a RESTful API for a social media platform?")
                    Ask the questions one at a time and wait for the candidate's response before proceeding to the next question. Randomize question types(conceptual, coding, debugging, application) for each technology.
                6.  **Context Handling:** Maintain context by referring to previous answers and using appropriate pronouns. If the candidate's response is unclear, ask for clarifications: "Could you elaborate on that?"
                7.  **Fallback Mechanism:** If you don't understand the candidate's input or if you get vague or incomplete response, ask them to rephrase their response. Example: "I'm sorry, I didn't understand that. Could you please rephrase your response?" Do not deviate from the interview purpose.
                8.  **End Conversation:** Upon completion of the technical questions or encountering a conversation-ending keyword like "exit," "quit," or "end," thank the candidate for their time and inform them that they will be contacted later. Example: "Thank you for your time. We will review your responses and contact you soon regarding the next steps."
                9.  **Handling Irrelevant or Rude Behavior:** If the candidate provides repeatedly irrelevant or off-topic responses, politely redirect them to the interview. If they are uncooperative or rude, terminate the interview. Example: "Thank you for your time. We will review your responses and contact you soon regarding the next steps."
                10.  **Goodbye:** Always conclude with "Goodbye."

                **Data Handling:** Handle all candidate data securely and in compliance with GDPR guidelines. Do not store sensitive information within the prompt itself.

                **Example (Python and Django):**

                If the candidate lists Python and Django, you might ask:

                *   (Conceptual - Python): "What are decorators in Python and how are they used?"
                *   (Coding Challenge - Python): "Write a Python function to check if a given string is a palindrome."
                *   (Conceptual - Django): "Explain the difference between `get()` and `filter()` methods in Django ORM."
                *   (Application - Django): "Describe how you would implement user authentication in a Django project.
                """
            }]
            
            # Send initial message to start the interview
            initial_response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=self.messages
            )
            self.messages.append({
                "role": "assistant", 
                "content": initial_response.choices[0].message.content
            })
            
            return "🎉 Bot initialized successfully! Starting the interview process..."
        except Exception as e:
            return f"❌ Error initializing bot: {str(e)}"  # Return error message if initialization fails

    def chat(self, message, chat_history):
        """Process a user message and return the chatbot's response.

        Args:
            message (str): The user's message to be processed.
            chat_history (list): The history of the chat conversation.

        Returns:
            str: The response from the chatbot or an error message.
        """
        try:
            if self.client is None:
                # Check if the bot is initialized
                return "⚠️ Please initialize the bot with your API key first."
            
            if self.moderator(message):
                # Check if the message violates content guidelines
                return "⚠️ I apologize, but I cannot process messages that violate content guidelines. Please rephrase your message appropriately."
            
            if "exit" in message.lower() or "goodbye" in message.lower() or "end" in message.lower() or "terminate" in message.lower():
                # Handle termination phrases
                return "👋 Thank you for your time! The interview process has ended. You will be contacted later."
                
            # Add user message to context
            self.messages.append({"role": "user", "content": message})
            
            # Get completion response from OpenAI
            completion = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=self.messages
            )
            
            # Extract the assistant's response
            assistant_message = completion.choices[0].message.content
            
            # Add assistant response to context
            self.messages.append({"role": "assistant", "content": assistant_message})
            
            return assistant_message
            
        except Exception as e:
            # Handle any exceptions that occur during message processing
            return f"❌ Error processing message: {str(e)}"

def create_interface():
    """Create the Gradio interface for the AI Technical Interviewer chatbot."""
    bot = ChatBot()

    # Updated custom CSS for a sleek design
    custom_css = """
    :root {
        --primary-color: #4a90e2;
        --secondary-color: #f0f2f5;
        --accent-color: #56cfe1;
        --text-color: #333333;
        --border-color: #dcdfe6;
        --background-gradient: linear-gradient(135deg, #4a90e2, #56cfe1);
    }

    body {
        background: var(--background-gradient);
        font-family: "Inter", sans-serif;
        color: var(--text-color);
        margin: 0;
        padding: 0;
    }

    .container {
        max-width: 800px;
        margin: 20px auto;
        padding: 20px;
        background: #ffffff;
        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.1);
        border-radius: 12px;
    }

    .header-container {
        text-align: center;
        margin-bottom: 20px;
    }

    .header-text {
        font-size: 2.5rem;
        font-weight: bold;
        color: var(--primary-color);
    }

    .subheader-text {
        font-size: 1rem;
        color: var(--text-color);
        margin-top: 10px;
    }

    .button-primary {
        background: var(--primary-color);
        color: #ffffff;
        border: none;
        padding: 10px 20px;
        border-radius: 8px;
        cursor: pointer;
        transition: background 0.3s ease;
    }

    .button-primary:hover {
        background: var(--accent-color);
    }

    .message-container {
        background: var(--secondary-color);
        padding: 20px;
        border-radius: 12px;
        max-height: 500px;
        overflow-y: auto;
        box-shadow: inset 0 0 10px rgba(0, 0, 0, 0.05);
    }

    .input-container {
        border: 1px solid var(--border-color);
        padding: 10px;
        border-radius: 8px;
        width: 100%;
    }

    .tips-section {
        margin-top: 20px;
        padding: 15px;
        background: var(--secondary-color);
        border-radius: 12px;
        font-size: 0.9rem;
    }

    """

    with gr.Blocks(css=custom_css, theme=gr.themes.Default(
        primary_hue="blue",
        secondary_hue="blue",
        neutral_hue="gray",
        font=["Inter", "-apple-system", "system-ui", "sans-serif"]
    )) as interface:
        with gr.Column(elem_classes="container"):
            # Header section
            with gr.Column(elem_classes="header-container"):
                gr.Markdown(
                    "AI Technical Interviewer",
                    elem_classes="header-text"
                )
                gr.Markdown(
                    """Transform your interview experience with our AI-powered technical assessment platform. 
                    Get ready for an intelligent, adaptive conversation that evaluates your skills effectively.""",
                    elem_classes="subheader-text"
                )

            # API section
            with gr.Column(elem_classes="api-section"):
                with gr.Row():
                    api_key_input = gr.Textbox(
                        label="OpenAI API Key",
                        placeholder="sk-...",
                        type="password",
                        scale=4
                    )
                    init_button = gr.Button(
                        "⚡ Initialize",
                        elem_classes="button-primary",
                        scale=1
                    )
                init_message = gr.Markdown(
                    visible=False,
                    elem_classes="init-message",
                    value=""
                )

            # Chat section
            with gr.Column(elem_classes="chatbot-container"):
                chatbot = gr.Chatbot(
                    value=[],
                    height=500,
                    show_label=False,
                    elem_classes="message-container"
                )
                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Type your message here...",
                        show_label=False,
                        elem_classes="input-container",
                        scale=4
                    )
                    send = gr.Button(
                        "Send",
                        elem_classes="button-primary",
                        scale=1
                    )

            # Tips section
            with gr.Column(elem_classes="tips-section"):
                gr.Markdown("""
                    💡 **Interview Tips**
                    - Be specific about your technical experience
                    - Take time to structure your responses
                    - Ask for clarification when needed
                    - Use examples to demonstrate your skills
                """)

            def user_message(user_msg, history):
                """Handle user message submission."""
                return "", history + [[user_msg, None]]

            def bot_message(history):
                """Generate bot response based on user message."""
                user_msg = history[-1][0]
                bot_response = bot.chat(user_msg, history)
                history[-1][1] = bot_response
                return history

            def init_and_start(api_key):
                """Initialize the bot and start the conversation."""
                init_result = bot.initialize_bot(api_key)
                if "successfully" in init_result:
                    # Get the first bot message
                    first_message = bot.messages[-1]["content"] if bot.messages else "Hello! Let's start the interview."
                    return init_result, [(None, first_message)]
                return init_result, []

            # Set up event handlers for user interactions
            msg.submit(user_message, [msg, chatbot], [msg, chatbot]).then(
                bot_message, chatbot, chatbot
            )
            send.click(user_message, [msg, chatbot], [msg, chatbot]).then(
                bot_message, chatbot, chatbot
            )

            init_button.click(
                fn=init_and_start,
                inputs=[api_key_input],
                outputs=[init_message, chatbot]
            ).then(
                lambda: gr.update(visible=True),
                None,
                [init_message]
            )

    return interface

if __name__ == "__main__":
    """Entry point for launching the Gradio interface."""
    interface = create_interface()  # Create the Gradio interface for the chatbot
    interface.launch()  # Launch the interface to start the application

c:\Users\sandy\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
